In [82]:
#数据预处理
# -*- coding: utf-8 -*-
"""
数据集准备脚本
"""
import os
import codecs
import shutil
try:
    import moxing as mox
except:
    print('not use moxing')
from glob import glob
from sklearn.model_selection import StratifiedShuffleSplit


def prepare_data_on_modelarts(args):
    """
    如果数据集存储在OBS，则需要将OBS上的数据拷贝到 ModelArts 中
    """
    # Create some local cache directories used for transfer data between local path and OBS path
    if not args.data_url.startswith('s3://'):
        args.data_local = args.data_url
    else:
        args.data_local = os.path.join(args.local_data_root, 'train_val')
        if not os.path.exists(args.data_local):
            mox.file.copy_parallel(args.data_url, args.data_local)
        else:
            print('args.data_local: %s is already exist, skip copy' % args.data_local)

    if not args.train_url.startswith('s3://'):
        args.train_local = args.train_url
    else:
        args.train_local = os.path.join(args.local_data_root, 'model_snapshots')
        if not os.path.exists(args.train_local):
            os.mkdir(args.train_local)

    if not args.test_data_url.startswith('s3://'):
        args.test_data_local = args.test_data_url
    else:
        args.test_data_local = os.path.join(args.local_data_root, 'test_data/')
        if not os.path.exists(args.test_data_local):
            mox.file.copy_parallel(args.test_data_url, args.test_data_local)
        else:
            print('args.test_data_local: %s is already exist, skip copy' % args.test_data_local)

    args.tmp = os.path.join(args.local_data_root, 'tmp')
    if not os.path.exists(args.tmp):
        os.mkdir(args.tmp)

    return args


def split_train_val(input_dir, output_train_dir, output_val_dir):
    """
    大赛发布的公开数据集是所有图片和标签txt都在一个目录中的格式
    如果需要使用 torch.utils.data.DataLoader 来加载数据，则需要将数据的存储格式做如下改变：
    1）划分训练集和验证集，分别存放为 train 和 val 目录；
    2）train 和 val 目录下有按类别存放的子目录，子目录中都是同一个类的图片
    本函数就是实现如上功能，建议先在自己的机器上运行本函数，然后将处理好的数据上传到OBS
    """
    if not os.path.exists(input_dir):
        print(input_dir, 'is not exist')
        return

    # 1. 检查图片和标签的一一对应
    label_file_paths = glob(os.path.join(input_dir, '*.txt'))
    valid_img_names = []
    valid_labels = []
    for file_path in label_file_paths:
        with codecs.open(file_path, 'r', 'utf-8') as f:
            line = f.readline()
        line_split = line.strip().split(', ')
        img_name = line_split[0]
        label_id = line_split[1]
        if os.path.exists(os.path.join(input_dir, img_name)):
            valid_img_names.append(img_name)
            valid_labels.append(int(label_id))
        else:
            print('error', img_name, 'is not exist')

    # 2. 使用 StratifiedShuffleSplit 划分训练集和验证集，可保证划分后各类别的占比保持一致
    # TODO，数据集划分方式可根据您的需要自行调整
    sss = StratifiedShuffleSplit(n_splits=1, test_size=500, random_state=0)
    sps = sss.split(valid_img_names, valid_labels)
    for sp in sps:
        train_index, val_index = sp

    label_id_name_dict = \
        {
            "0": "工艺品/仿唐三彩",
            "1": "工艺品/仿宋木叶盏",
            "2": "工艺品/布贴绣",
            "3": "工艺品/景泰蓝",
            "4": "工艺品/木马勺脸谱",
            "5": "工艺品/柳编",
            "6": "工艺品/葡萄花鸟纹银香囊",
            "7": "工艺品/西安剪纸",
            "8": "工艺品/陕历博唐妞系列",
            "9": "景点/关中书院",
            "10": "景点/兵马俑",
            "11": "景点/南五台",
            "12": "景点/大兴善寺",
            "13": "景点/大观楼",
            "14": "景点/大雁塔",
            "15": "景点/小雁塔",
            "16": "景点/未央宫城墙遗址",
            "17": "景点/水陆庵壁塑",
            "18": "景点/汉长安城遗址",
            "19": "景点/西安城墙",
            "20": "景点/钟楼",
            "21": "景点/长安华严寺",
            "22": "景点/阿房宫遗址",
            "23": "民俗/唢呐",
            "24": "民俗/皮影",
            "25": "特产/临潼火晶柿子",
            "26": "特产/山茱萸",
            "27": "特产/玉器",
            "28": "特产/阎良甜瓜",
            "29": "特产/陕北红小豆",
            "30": "特产/高陵冬枣",
            "31": "美食/八宝玫瑰镜糕",
            "32": "美食/凉皮",
            "33": "美食/凉鱼",
            "34": "美食/德懋恭水晶饼",
            "35": "美食/搅团",
            "36": "美食/枸杞炖银耳",
            "37": "美食/柿子饼",
            "38": "美食/浆水面",
            "39": "美食/灌汤包",
            "40": "美食/烧肘子",
            "41": "美食/石子饼",
            "42": "美食/神仙粉",
            "43": "美食/粉汤羊血",
            "44": "美食/羊肉泡馍",
            "45": "美食/肉夹馍",
            "46": "美食/荞面饸饹",
            "47": "美食/菠菜面",
            "48": "美食/蜂蜜凉粽子",
            "49": "美食/蜜饯张口酥饺",
            "50": "美食/西安油茶",
            "51": "美食/贵妃鸡翅",
            "52": "美食/醪糟",
            "53": "美食/金线油塔"
        }

    # 3. 创建 output_train_dir 目录下的所有标签名子目录
    for id in label_id_name_dict.keys():
        if not os.path.exists(os.path.join(output_train_dir, id)):
            os.mkdir(os.path.join(output_train_dir, id))

    # 4. 将训练集图片拷贝到 output_train_dir 目录
    for index in train_index:
        file_path = label_file_paths[index]
        with codecs.open(file_path, 'r', 'utf-8') as f:
            gt_label = f.readline()
        img_name = gt_label.split(',')[0].strip()
        id = gt_label.split(',')[1].strip()
        shutil.copy(os.path.join(input_dir, img_name), os.path.join(output_train_dir, id, img_name))

    # 5. 创建 output_val_dir 目录下的所有标签名子目录
    for id in label_id_name_dict.keys():
        if not os.path.exists(os.path.join(output_val_dir, id)):
            os.mkdir(os.path.join(output_val_dir, id))

    # 6. 将验证集图片拷贝到 output_val_dir 目录
    for index in val_index:
        file_path = label_file_paths[index]
        with codecs.open(file_path, 'r', 'utf-8') as f:
            gt_label = f.readline()
        img_name = gt_label.split(',')[0].strip()
        id = gt_label.split(',')[1].strip()
        shutil.copy(os.path.join(input_dir, img_name), os.path.join(output_val_dir, id, img_name))

    print('total samples: %d, train samples: %d, val samples:%d'
          % (len(valid_labels), len(train_index), len(val_index)))
    print('end')


if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser(description='data prepare')
    parser.add_argument('--input_dir', required=True, type=str, help='input data dir')
    parser.add_argument('--output_train_dir', required=True, type=str, help='output train data dir')
    parser.add_argument('--output_val_dir', required=True, type=str, help='output validation data dir')
    args = parser.parse_args()
    if args.input_dir == '' or args.output_train_dir == '' or args.output_val_dir == '':
        raise Exception('You must specify valid arguments')
    if not os.path.exists(args.output_train_dir):
        os.makedirs(args.output_train_dir)
    if not os.path.exists(args.output_val_dir):
        os.makedirs(args.output_val_dir)
    split_train_val(args.input_dir, args.output_train_dir, args.output_val_dir)

not use moxing


D:\Anaconada\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconada\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconada\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconada\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconada\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **k

SystemExit: 2

D:\Anaconada\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#主函数
# -*- coding: utf-8 -*-
"""
基于 PyTorch resnet50 实现的图片分类代码
原代码地址：https://github.com/pytorch/examples/blob/master/imagenet/main.py
可以与原代码进行比较，查看需修改哪些代码才可以将其改造成可以在 ModelArts 上运行的代码
在ModelArts Notebook中的代码运行方法：
（0）准备数据
大赛发布的公开数据集是所有图片和标签txt都在一个目录中的格式
如果需要使用 torch.utils.data.DataLoader 来加载数据，则需要将数据的存储格式做如下改变：
1）划分训练集和验证集，分别存放为 train 和 val 目录；
2）train 和 val 目录下有按类别存放的子目录，子目录中都是同一个类的图片
prepare_data.py中的 split_train_val 函数就是实现如上功能，建议先在自己的机器上运行该函数，然后将处理好的数据上传到OBS
执行该函数的方法如下：
cd {prepare_data.py所在目录}
python prepare_data.py --input_dir '../datasets/train_data' --output_train_dir '../datasets/train_val/train' --output_val_dir '../datasets/train_val/val'

（1）从零训练
cd {main.py所在目录}
python main.py --data_url '../datasets/train_val' --train_url '../model_snapshots' --deploy_script_path './deploy_scripts' --arch 'resnet50' --num_classes 54 --workers 4 --epochs 6 --pretrained True --seed 0

（2）加载已有模型继续训练
cd {main.py所在目录}
python main.py --data_url '../datasets/train_val' --train_url '../model_snapshots' --deploy_script_path './deploy_scripts' --arch 'resnet50' --num_classes 54 --workers 4 --epochs 6 --seed 0 --resume '../model_snapshots/epoch_0_2.4.pth'

（3）评价单个pth文件
cd {main.py所在目录}
python main.py --data_url '../datasets/train_val' --train_url '../model_snapshots' --arch 'resnet50' --num_classes 54 --seed 0 --eval_pth '../model_snapshots/epoch_5_8.4.pth'
"""
import argparse
import os
import random
import shutil
import time
import warnings

try:
    import moxing as mox
except:
    print('not use moxing')
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from prepare_data import prepare_data_on_modelarts

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
# parser.add_argument('data', metavar='DIR',
#                     help='path to dataset')
parser.add_argument('-a', '--arch', metavar='ARCH', required=True,
                    choices=model_names,
                    help='model architecture: ' +
                        ' | '.join(model_names) +
                        ' (default: resnet18)')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=10, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=128, type=int,
                    metavar='N',
                    help='mini-batch size (default: 128), this is the total '
                         'batch size of all GPUs on the current node when '
                         'using Data Parallel or Distributed Data Parallel')
parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                    metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--wd', '--weight-decay', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)',
                    dest='weight_decay')
parser.add_argument('-p', '--print_freq', default=5, type=int,
                    metavar='N', help='print frequency (default: 10)')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
# parser.add_argument('-e', '--evaluate', dest='evaluate', action='store_true',
#                     help='evaluate model on validation set')
parser.add_argument('--eval_pth', default='', type=str,
                    help='the *.pth model path need to be evaluated on validation set')
parser.add_argument('--pretrained', default=False, type=bool,
                    help='use pre-trained model or not')
parser.add_argument('--world-size', default=-1, type=int,
                    help='number of nodes for distributed training')
parser.add_argument('--rank', default=-1, type=int,
                    help='node rank for distributed training')
parser.add_argument('--dist-url', default='tcp://224.66.41.62:23456', type=str,
                    help='url used to set up distributed training')
parser.add_argument('--dist-backend', default='nccl', type=str,
                    help='distributed backend')
parser.add_argument('--seed', default=None, type=int,
                    help='seed for initializing training. ')
parser.add_argument('--gpu', default=None, type=int,
                    help='GPU id to use.')
parser.add_argument('--multiprocessing-distributed', action='store_true',
                    help='Use multi-processing distributed training to launch '
                         'N processes per node, which has N GPUs. This is the '
                         'fastest way to use PyTorch for either single node or '
                         'multi node data parallel training')

# These arguments are added for adapting ModelArts
parser.add_argument('--num_classes', required=True, type=int, help='the num of classes which your task should classify')
parser.add_argument('--local_data_root', default='/cache/', type=str,
                    help='a directory used for transfer data between local path and OBS path')
parser.add_argument('--data_url', required=True, type=str, help='the training and validation data path')
parser.add_argument('--test_data_url', default='', type=str, help='the test data path')
parser.add_argument('--data_local', default='', type=str, help='the training and validation data path on local')
parser.add_argument('--test_data_local', default='', type=str, help='the test data path on local')
parser.add_argument('--train_url', required=True, type=str, help='the path to save training outputs')
parser.add_argument('--train_local', default='', type=str, help='the training output results on local')
parser.add_argument('--tmp', default='', type=str, help='a temporary path on local')
parser.add_argument('--deploy_script_path', default='', type=str,
                    help='a path which contain config.json and customize_service.py, '
                         'if it is set, these two scripts will be copied to {train_url}/model directory')
best_acc1 = 0


def main():
    args, unknown = parser.parse_known_args()
    args = prepare_data_on_modelarts(args)

    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True
        warnings.warn('You have chosen to seed training. '
                      'This will turn on the CUDNN deterministic setting, '
                      'which can slow down your training considerably! '
                      'You may see unexpected behavior when restarting '
                      'from checkpoints.')

    if args.gpu is not None:
        warnings.warn('You have chosen a specific GPU. This will completely '
                      'disable data parallelism.')

    if args.dist_url == "env://" and args.world_size == -1:
        args.world_size = int(os.environ["WORLD_SIZE"])

    args.distributed = args.world_size > 1 or args.multiprocessing_distributed

    ngpus_per_node = torch.cuda.device_count()
    if args.multiprocessing_distributed:
        # Since we have ngpus_per_node processes per node, the total world_size
        # needs to be adjusted accordingly
        args.world_size = ngpus_per_node * args.world_size
        # Use torch.multiprocessing.spawn to launch distributed processes: the
        # main_worker process function
        mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))
    else:
        # Simply call main_worker function
        main_worker(args.gpu, ngpus_per_node, args)


def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            # For multiprocessing distributed training, rank needs to be the
            # global rank among all the processes
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)
    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        os.environ['TORCH_MODEL_ZOO'] = '../pre-trained_model/pytorch'
        if not mox.file.exists('../pre-trained_model/pytorch/resnet50-19c8e357.pth'):
            mox.file.copy('s3://ma-competitions-bj4/model_zoo/pytorch/resnet50-19c8e357.pth',
                          '../pre-trained_model/pytorch/resnet50-19c8e357.pth')
            print('copy pre-trained model from OBS to: %s success' %
                  (os.path.abspath('../pre-trained_model/pytorch/resnet50-19c8e357.pth')))
        else:
            print('use exist pre-trained model at: %s' %
                  (os.path.abspath('../pre-trained_model/pytorch/resnet50-19c8e357.pth')))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.arch))
        model = models.__dict__[args.arch]()

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, args.num_classes)

    if args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            model.cuda(args.gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        else:
            model.cuda()
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model = model.cuda(args.gpu)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # optionally resume from a checkpoint
    if args.resume:
        # if os.path.isfile(args.resume):
        if mox.file.exists(args.resume) and (not mox.file.is_directory(args.resume)):
            if args.resume.startswith('s3://'):
                restore_model_name = args.resume.rsplit('/', 1)[1]
                mox.file.copy(args.resume, '/cache/tmp/' + restore_model_name)
                args.resume = '/cache/tmp/' + restore_model_name
            print("=> loading checkpoint '{}'".format(args.resume))
            if args.gpu is None:
                checkpoint = torch.load(args.resume)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(args.gpu)
                checkpoint = torch.load(args.resume, map_location=loc)
            if args.resume.startswith('/cache/tmp/'):
                os.remove(args.resume)

            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = os.path.join(args.data_local, 'train')
    valdir = os.path.join(args.data_local, 'val')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    if args.eval_pth != '':
        if mox.file.exists(args.eval_pth) and (not mox.file.is_directory(args.eval_pth)):
            if args.eval_pth.startswith('s3://'):
                model_name = args.eval_pth.rsplit('/', 1)[1]
                mox.file.copy(args.eval_pth, '/cache/tmp/' + model_name)
                args.eval_pth = '/cache/tmp/' + model_name
            print("=> loading checkpoint '{}'".format(args.eval_pth))
            if args.gpu is None:
                checkpoint = torch.load(args.eval_pth)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(args.gpu)
                checkpoint = torch.load(args.eval_pth, map_location=loc)
            if args.eval_pth.startswith('/cache/tmp/'):
                os.remove(args.eval_pth)

            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.eval_pth, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.eval_pth))

        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)

        # evaluate on validation set
        if epoch % args.print_freq == 0:
            acc1 = validate(val_loader, model, criterion, args)

            # remember best acc@1 and save checkpoint
            is_best = False
            best_acc1 = max(acc1.item(), best_acc1)
            pth_file_name = os.path.join(args.train_local, 'epoch_%s_%s.pth'
                                         % (str(epoch), str(round(acc1.item(), 3))))
            if not args.multiprocessing_distributed or (args.multiprocessing_distributed
                    and args.rank % ngpus_per_node == 0):
                save_checkpoint({
                    'epoch': epoch,
                    'arch': args.arch,
                    'state_dict': model.state_dict(),
                    'best_acc1': best_acc1,
                    'optimizer': optimizer.state_dict(),
                }, is_best, pth_file_name, args)

    if args.epochs >= args.print_freq:
        save_best_checkpoint(best_acc1, args)


def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        target = target.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if args.gpu is not None:
                images = images.cuda(args.gpu, non_blocking=True)
            target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


def save_checkpoint(state, is_best, filename, args):
    if not is_best:
        torch.save(state, filename)
        if args.train_url.startswith('s3'):
            mox.file.copy(filename,
                          args.train_url + '/' + os.path.basename(filename))
            os.remove(filename)


def save_best_checkpoint(best_acc1, args):
    best_acc1_suffix = '%s.pth' % str(round(best_acc1, 3))
    pth_files = mox.file.list_directory(args.train_url)
    for pth_name in pth_files:
        if pth_name.endswith(best_acc1_suffix):
            break

    # mox.file可兼容处理本地路径和OBS路径
    if not mox.file.exists(os.path.join(args.train_url, 'model')):
        mox.file.mk_dir(os.path.join(args.train_url, 'model'))

    mox.file.copy(os.path.join(args.train_url, pth_name), os.path.join(args.train_url, 'model/model_best.pth'))
    mox.file.copy(os.path.join(args.deploy_script_path, 'config.json'),
                  os.path.join(args.train_url, 'model/config.json'))
    mox.file.copy(os.path.join(args.deploy_script_path, 'customize_service.py'),
                  os.path.join(args.train_url, 'model/customize_service.py'))
    if mox.file.exists(os.path.join(args.train_url, 'model/config.json')) and \
            mox.file.exists(os.path.join(args.train_url, 'model/customize_service.py')):
        print('copy config.json and customize_service.py success')
    else:
        print('copy config.json and customize_service.py failed')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


if __name__ == '__main__':
    main()

In [ ]:
#inference
# -*- coding: utf-8 -*-
import os
import codecs
import numpy as np
from PIL import Image
from collections import OrderedDict

import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
# from model_service.pytorch_model_service import PTServingBaseService
#
# import time
# from metric.metrics_manager import MetricsManager
# import log
# logger = log.getLogger(__name__)


class ImageClassificationService():
    def __init__(self, model_name, model_path):
        self.model_name = model_name
        self.model_path = model_path

        self.model = models.__dict__['resnet50'](num_classes=54)
        self.use_cuda = False
        if torch.cuda.is_available():
            print('Using GPU for inference')
            self.use_cuda = True
            checkpoint = torch.load(self.model_path)
            self.model = torch.nn.DataParallel(self.model).cuda()
            self.model.load_state_dict(checkpoint['state_dict'])
        else:
            print('Using CPU for inference')
            checkpoint = torch.load(self.model_path, map_location='cpu')
            state_dict = OrderedDict()
            # 训练脚本 main.py 中保存了'epoch', 'arch', 'state_dict', 'best_acc1', 'optimizer'五个key值，
            # 其中'state_dict'对应的value才是模型的参数。
            # 训练脚本 main.py 中创建模型时用了torch.nn.DataParallel，因此模型保存时的dict都会有‘module.’的前缀，
            # 下面 tmp = key[7:] 这行代码的作用就是去掉‘module.’前缀
            for key, value in checkpoint['state_dict'].items():
                tmp = key[7:]
                state_dict[tmp] = value
            self.model.load_state_dict(state_dict)

        # self.model.eval()

        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

        self.transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            self.normalize
        ])

        self.label_id_name_dict = \
            {
                "0": "工艺品/仿唐三彩",
                "1": "工艺品/仿宋木叶盏",
                "2": "工艺品/布贴绣",
                "3": "工艺品/景泰蓝",
                "4": "工艺品/木马勺脸谱",
                "5": "工艺品/柳编",
                "6": "工艺品/葡萄花鸟纹银香囊",
                "7": "工艺品/西安剪纸",
                "8": "工艺品/陕历博唐妞系列",
                "9": "景点/关中书院",
                "10": "景点/兵马俑",
                "11": "景点/南五台",
                "12": "景点/大兴善寺",
                "13": "景点/大观楼",
                "14": "景点/大雁塔",
                "15": "景点/小雁塔",
                "16": "景点/未央宫城墙遗址",
                "17": "景点/水陆庵壁塑",
                "18": "景点/汉长安城遗址",
                "19": "景点/西安城墙",
                "20": "景点/钟楼",
                "21": "景点/长安华严寺",
                "22": "景点/阿房宫遗址",
                "23": "民俗/唢呐",
                "24": "民俗/皮影",
                "25": "特产/临潼火晶柿子",
                "26": "特产/山茱萸",
                "27": "特产/玉器",
                "28": "特产/阎良甜瓜",
                "29": "特产/陕北红小豆",
                "30": "特产/高陵冬枣",
                "31": "美食/八宝玫瑰镜糕",
                "32": "美食/凉皮",
                "33": "美食/凉鱼",
                "34": "美食/德懋恭水晶饼",
                "35": "美食/搅团",
                "36": "美食/枸杞炖银耳",
                "37": "美食/柿子饼",
                "38": "美食/浆水面",
                "39": "美食/灌汤包",
                "40": "美食/烧肘子",
                "41": "美食/石子饼",
                "42": "美食/神仙粉",
                "43": "美食/粉汤羊血",
                "44": "美食/羊肉泡馍",
                "45": "美食/肉夹馍",
                "46": "美食/荞面饸饹",
                "47": "美食/菠菜面",
                "48": "美食/蜂蜜凉粽子",
                "49": "美食/蜜饯张口酥饺",
                "50": "美食/西安油茶",
                "51": "美食/贵妃鸡翅",
                "52": "美食/醪糟",
                "53": "美食/金线油塔"
            }

    def _preprocess(self, data):
        preprocessed_data = {}
        for k, v in data.items():
            for file_name, file_content in v.items():
                img = Image.open(file_content)
                img = self.transforms(img)
                preprocessed_data[k] = img
        return preprocessed_data

    def _inference(self, data):
        img = data["input_img"]
        img = img.unsqueeze(0)

        with torch.no_grad():
            pred_score = self.model(img)
            pred_score = F.softmax(pred_score.data, dim=1)
            if pred_score is not None:
                pred_label = torch.argsort(pred_score[0], descending=True)[:1][0].item()
                result = {'result': self.label_id_name_dict[str(pred_label)]}
            else:
                result = {'result': 'predict score is None'}

        return result

    def _postprocess(self, data):
        return data

    # def inference(self, data):
    #     """
    #     Wrapper function to run preprocess, inference and postprocess functions.
    #
    #     Parameters
    #     ----------
    #     data : map of object
    #         Raw input from request.
    #
    #     Returns
    #     -------
    #     list of outputs to be sent back to client.
    #         data to be sent back
    #     """
    #     pre_start_time = time.time()
    #     data = self._preprocess(data)
    #     infer_start_time = time.time()
    #
    #     # Update preprocess latency metric
    #     pre_time_in_ms = (infer_start_time - pre_start_time) * 1000
    #     logger.info('preprocess time: ' + str(pre_time_in_ms) + 'ms')
    #
    #     if self.model_name + '_LatencyPreprocess' in MetricsManager.metrics:
    #         MetricsManager.metrics[self.model_name + '_LatencyPreprocess'].update(pre_time_in_ms)
    #
    #     data = self._inference(data)
    #     infer_end_time = time.time()
    #     infer_in_ms = (infer_end_time - infer_start_time) * 1000
    #
    #     logger.info('infer time: ' + str(infer_in_ms) + 'ms')
    #     data = self._postprocess(data)
    #
    #     # Update inference latency metric
    #     post_time_in_ms = (time.time() - infer_end_time) * 1000
    #     logger.info('postprocess time: ' + str(post_time_in_ms) + 'ms')
    #     if self.model_name + '_LatencyInference' in MetricsManager.metrics:
    #         MetricsManager.metrics[self.model_name + '_LatencyInference'].update(post_time_in_ms)
    #
    #     # Update overall latency metric
    #     if self.model_name + '_LatencyOverall' in MetricsManager.metrics:
    #         MetricsManager.metrics[self.model_name + '_LatencyOverall'].update(pre_time_in_ms + post_time_in_ms)
    #
    #     logger.info('latency: ' + str(pre_time_in_ms + infer_in_ms + post_time_in_ms) + 'ms')
    #     data['latency_time'] = pre_time_in_ms + infer_in_ms + post_time_in_ms
    #     time.sleep(1)
    #     return data


def infer_on_dataset(img_dir, label_dir, model_path):
    if not os.path.exists(img_dir):
        print('img_dir: %s is not exist' % img_dir)
        return None
    if not os.path.exists(label_dir):
        print('label_dir: %s is not exist' % label_dir)
        return None
    if not os.path.exists(model_path):
        print('model_path: %s is not exist' % model_path)
        return None
    output_dir = model_path + '_output'
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    infer = ImageClassificationService('', model_path)
    files = os.listdir(img_dir)
    error_results = []
    right_count = 0
    total_count = 0
    for file_name in files:
        if not file_name.endswith('jpg'):
            continue

        with codecs.open(os.path.join(label_dir, file_name.split('.jpg')[0] + '.txt'), 'r', 'utf-8') as f:
            line = f.readline()
        line_split = line.strip().split(', ')
        if len(line_split) != 2:
            print('%s contain error lable' % os.path.basename(file_name.split('.jpg')[0] + '.txt'))
            continue
        gt_label = infer.label_id_name_dict[line_split[1]]
        # gt_label = "工艺品/仿唐三彩"

        img_path = os.path.join(img_dir, file_name)
        img = Image.open(img_path)
        img = infer.transforms(img)
        result = infer._inference({"input_img": img})
        pred_label = result.get('result', 'error')

        total_count += 1
        if pred_label == gt_label:
            right_count += 1
        else:
            error_results.append(', '.join([file_name, gt_label, pred_label]) + '\n')

    acc = float(right_count) / total_count
    result_file_path = os.path.join(output_dir, 'accuracy.txt')
    with codecs.open(result_file_path, 'w', 'utf-8') as f:
        f.write('# predict error files\n')
        f.write('####################################\n')
        f.write('file_name, gt_label, pred_label\n')
        f.writelines(error_results)
        f.write('####################################\n')
        f.write('accuracy: %s\n' % acc)
    print('accuracy result file saved as %s' % result_file_path)
    print('accuracy: %0.4f' % acc)
    return acc, result_file_path


if __name__ == '__main__':
    img_dir = r'/home/ma-user/work/xi_an_ai/datasets/test_data'
    label_dir = r'/home/ma-user/work/xi_an_ai/datasets/test_data'
    model_path = r'/home/ma-user/work/xi_an_ai/model_snapshots/pytorch/V0001/model/model_best.pth'
    infer_on_dataset(img_dir, label_dir, model_path)

In [ ]:
import matplotlib.pyplot as plt
import random
import cv2
import numpy as np
data_x = np.zeros((1,224*224*3))
data_label = np.zeros((1,1))
def read_and_argumentation(data_x,data_label,label_id_name_dict):
    k = random.sample(list(range(4794)),4793)
    progress = 0
    for i in k:
        if i==0:
            continue
        try:
            data_0 = cv2.imread('C:/Users/guyue/Desktop/train_data/img_'+str(i)+'.jpg')
            data_0 = cv2.resize(data_0,(224,224),interpolation = cv2.INTER_CUBIC)
            data_y = open('C:/Users/guyue/Desktop/train_data/img_'+str(i)+'.txt','r')
            test = data_y.read()
            if test=='':
                data_y = np.array(test[-1],np.float32)
            else:
                data_y = np.array(test[-2:],np.float32)
            data_x_i,data_y_i = argumentation(data_0,data_y)
            data_x = np.vstack((data_x,data_x_i.reshape(-1,224*224*3)))
            data_label = np.vstack((data_label,data_y_i))
            progress+=1
            print('第'+str(progress)+'已完成')
        except:
            continue
    return (data_x/255.)[1:,:],data_label[1:]
def argumentation(data_x,data_y):
    data_x = data_x.reshape(224,224,3)
    rot_mat = cv2.getRotationMatrix2D((data_x.shape[0]/2,data_x.shape[1]/2),90,1)
    data_x_1 = cv2.warpAffine(data_0, rot_mat, (data_0.shape[1], data_0.shape[0]))
    data_x_2 = cv2.GaussianBlur(data_x,(5,5),0)
    data_y = np.vstack((data_y,data_y,data_y))
    data_x = np.vstack((data_x.reshape(1,224*224*3),data_x_1.reshape(1,224*224*3),data_x_2.reshape(1,224*224*3)))
    return data_x,data_y
data_x_,data_y_ = read_and_argumentation(data_x,data_label,label_id_name_dict)
data_all = np.hstack((data_x_,data_y_))
np.random.shuffle(data_all)
data_x,data_y = data_all[:,:-1],data_all[:,-1]
label_id_name_dict = \
            {
                "0": "工艺品/仿唐三彩",
                "1": "工艺品/仿宋木叶盏",
                "2": "工艺品/布贴绣",
                "3": "工艺品/景泰蓝",
                "4": "工艺品/木马勺脸谱",
                "5": "工艺品/柳编",
                "6": "工艺品/葡萄花鸟纹银香囊",
                "7": "工艺品/西安剪纸",
                "8": "工艺品/陕历博唐妞系列",
                "9": "景点/关中书院",
                "10": "景点/兵马俑",
                "11": "景点/南五台",
                "12": "景点/大兴善寺",
                "13": "景点/大观楼",
                "14": "景点/大雁塔",
                "15": "景点/小雁塔",
                "16": "景点/未央宫城墙遗址",
                "17": "景点/水陆庵壁塑",
                "18": "景点/汉长安城遗址",
                "19": "景点/西安城墙",
                "20": "景点/钟楼",
                "21": "景点/长安华严寺",
                "22": "景点/阿房宫遗址",
                "23": "民俗/唢呐",
                "24": "民俗/皮影",
                "25": "特产/临潼火晶柿子",
                "26": "特产/山茱萸",
                "27": "特产/玉器",
                "28": "特产/阎良甜瓜",
                "29": "特产/陕北红小豆",
                "30": "特产/高陵冬枣",
                "31": "美食/八宝玫瑰镜糕",
                "32": "美食/凉皮",
                "33": "美食/凉鱼",
                "34": "美食/德懋恭水晶饼",
                "35": "美食/搅团",
                "36": "美食/枸杞炖银耳",
                "37": "美食/柿子饼",
                "38": "美食/浆水面",
                "39": "美食/灌汤包",
                "40": "美食/烧肘子",
                "41": "美食/石子饼",
                "42": "美食/神仙粉",
                "43": "美食/粉汤羊血",
                "44": "美食/羊肉泡馍",
                "45": "美食/肉夹馍",
                "46": "美食/荞面饸饹",
                "47": "美食/菠菜面",
                "48": "美食/蜂蜜凉粽子",
                "49": "美食/蜜饯张口酥饺",
                "50": "美食/西安油茶",
                "51": "美食/贵妃鸡翅",
                "52": "美食/醪糟",
                "53": "美食/金线油塔"
            }

第1已完成
第2已完成
第3已完成
第4已完成
第5已完成
第6已完成
第7已完成
第8已完成
第9已完成
第10已完成
第11已完成
第12已完成
第13已完成
第14已完成
第15已完成
第16已完成
第17已完成
第18已完成
第19已完成
第20已完成
第21已完成
第22已完成
第23已完成
第24已完成
第25已完成
第26已完成
第27已完成
第28已完成
第29已完成
第30已完成
第31已完成
第32已完成
第33已完成
第34已完成
第35已完成
第36已完成
第37已完成
第38已完成
第39已完成
第40已完成
第41已完成
第42已完成
第43已完成
第44已完成
第45已完成
第46已完成
第47已完成
第48已完成
第49已完成
第50已完成
第51已完成
第52已完成
第53已完成
第54已完成
第55已完成
第56已完成
第57已完成
第58已完成
第59已完成
第60已完成
第61已完成
第62已完成
第63已完成
第64已完成
第65已完成
第66已完成
第67已完成
第68已完成
第69已完成
第70已完成
第71已完成
第72已完成
第73已完成
第74已完成
第75已完成
第76已完成
第77已完成
第78已完成
第79已完成
第80已完成
第81已完成
第82已完成
第83已完成
第84已完成
第85已完成
第86已完成
第87已完成
第88已完成
第89已完成
第90已完成
第91已完成
第92已完成
第93已完成
第94已完成
第95已完成
第96已完成
第97已完成
第98已完成
第99已完成
第100已完成
第101已完成
第102已完成
第103已完成
第104已完成
第105已完成
第106已完成
第107已完成
第108已完成
第109已完成
第110已完成
第111已完成
第112已完成
第113已完成
第114已完成
第115已完成
第116已完成
第117已完成
第118已完成
第119已完成
第120已完成
第121已完成
第122已完成
第123已完成
第124已完成
第125已完成
第126已完成
第127已完成
第128已完成
第129已完成
第130已完成
第131已完成
第132已完成
第133已完成
第134已完成
第135已完成
第136已完成
第137已完成
第138已完成
第139

第1035已完成
第1036已完成
第1037已完成
第1038已完成
第1039已完成
第1040已完成
第1041已完成
第1042已完成
第1043已完成
第1044已完成
第1045已完成
第1046已完成
第1047已完成
第1048已完成
第1049已完成
第1050已完成
第1051已完成
第1052已完成
第1053已完成
第1054已完成
第1055已完成
第1056已完成
第1057已完成
第1058已完成
第1059已完成
第1060已完成
第1061已完成
第1062已完成
第1063已完成
第1064已完成
第1065已完成
第1066已完成
第1067已完成
第1068已完成
第1069已完成
第1070已完成
第1071已完成
第1072已完成
第1073已完成
第1074已完成
第1075已完成
第1076已完成
第1077已完成
第1078已完成
第1079已完成
第1080已完成
第1081已完成
第1082已完成
第1083已完成
第1084已完成
第1085已完成
第1086已完成
第1087已完成
第1088已完成
第1089已完成
第1090已完成
第1091已完成
第1092已完成
第1093已完成
第1094已完成
第1095已完成
第1096已完成
第1097已完成
第1098已完成
第1099已完成
第1100已完成
第1101已完成
第1102已完成
第1103已完成
第1104已完成
第1105已完成
第1106已完成
第1107已完成
第1108已完成
第1109已完成
第1110已完成
第1111已完成
第1112已完成
第1113已完成
第1114已完成
第1115已完成
第1116已完成
第1117已完成
第1118已完成
第1119已完成
第1120已完成
第1121已完成
第1122已完成
第1123已完成
第1124已完成
第1125已完成
第1126已完成
第1127已完成
第1128已完成
第1129已完成
第1130已完成
第1131已完成
第1132已完成
第1133已完成
第1134已完成
第1135已完成
第1136已完成
第1137已完成
第1138已完成
第1139已完成
第1140已完成
第1141已完成
第1142已完成
第1143已完成
第1144已完成
第1145已完成
第

In [85]:
import tensorflow as tf
import tensorflow.keras.layers
from sklearn.model_selection import train_test_split
import numpy as np

In [87]:
class nn:
    def __init__(self,data_x,data_y,train_layer_num=-2):
        '''
        data_x:数据，增强后，且大小为[-1,224*224*3]
        data_x:标签，大小为[-1,1]
        image_shape = [224,224,3]
        train_layer_num = 欲训练迁移模型参数层数，使用负值，默认为-2，即只训练全连接层与分类层
        '''
        self.model = tf.keras.applications.MobileNetV2(input_shape = (224,224,3),include_top=False,weights = 'imagenet')
        self.data_x = data_x
        self.data_y = data_y
        self.class_num  = 53
        self.test_accuracy = 0.
        self.train_layer_num = train_layer_num
    def build_model(self):
        self.model = tf.keras.Sequential([
            self.model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(2048,activation='relu'),
            tf.keras.layers.Dense(self.class_num,activation='softmax')
        ])
        for layer in self.model.layers[:self.train_layer_num]:
            layer.trainable = False
        self.model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])
        self.model.summary()
    def onehot(self):
        data_y_onehot = np.zeros((self.data_y.shape[0],self.class_num))
        for i in range(data_y_onehot.shape[0]):
            data_y_onehot[i,self.data_y[i]]=0
        return data_y_onehot
    def train_split_data(self):
        data_x_train,data_x_test,data_y_train,data_y_test = train_test_split(self.data_x,self.onehot(),test_size=0.3)
        return data_x_train,data_x_test,data_y_train,data_y_test
    def main(self):
        data_x_train,data_x_test,data_y_train,data_y_test = self.train_split_data()
        data_x_train = tf.reshape(tf.convert_to_tensor(data_x_train,tf.float32),[-1,224,224,3])
        data_x_test = tf.reshape(tf.convert_to_tensor(data_x_test,tf.float32),[-1,224,224,3])
        data_y_train = tf.convert_to_tensor(data_y_train,tf.float32)
        data_y_test = tf.convert_to_tensor(data_y_test,tf.float32)
        self.model.fit(data_x_train,data_y_train,epochs=50)
        self.test_accuracy = self.evaluate(data_x_test,data_y_test)
compute = nn(data_x,data_y)
compute.build_model()

In [93]:
random.sample(list(range(1,91)),1)

[67]

In [98]:
np.random.shuffle(np.arange(9).reshape(3,3))

None
